# Load packages + data

In [1]:
from IPython.core.magic import register_cell_magic 
from IPython.display import HTML, display

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as stats
import statsmodels.formula.api as smf

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score

from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
import tensorflow.keras as keras
from keras import Sequential
from tensorflow.keras.layers import Dense, Input

import pickle

pd.options.mode.chained_assignment = None

DATA_PATH = './data/'
DATA_FN   = "train.csv"

In [2]:
!ls

3001FinalProject    README.md           pred_prob_def
Final_Attempt.ipynb data


In [3]:
data = pd.read_csv(DATA_PATH+DATA_FN)
# print(data.shape)
# data

# Feature Extraction

In [4]:
# Aggregating terms
def feature_extraction_modified(data,relevantAttributes):
  current_liabilities = (data['debt_bank_st'] + data['debt_fin_st'])
  current_assets_minus_inventory = data['asst_current'] # For now, we are considering no inventory
  cash_mktbl_securities = current_assets_minus_inventory - data['AR']
  working_capital = data['asst_current'] - current_liabilities # Also used as an indicator by itself
  fixed_assets = data['asst_tang_fixed'] + data['asst_intang_fixed']

  # Adding Liquidity ratios
  # For other benchmarks, we can compare with the distribution of these ratios in the dataset (i.e. how many std deviations away from mean)

  m = 1
  current_ratio = (data['asst_current'] + m)/(data['asst_current'] + current_liabilities + m) 

  quick_ratio = (current_assets_minus_inventory + m)/(current_assets_minus_inventory + current_liabilities + m) 

  cash_ratio  = (cash_mktbl_securities + m)/(cash_mktbl_securities + current_liabilities + m)

  # Operating Cash Flow Ratio
  cfo_ratio = (data['cf_operations'] + m)/(data['cf_operations'] + current_liabilities + m) 


  # Adding Activity ratios

  # Net revenue is net sales (according to the internet)

  working_capital_turnover = (data['rev_operating'] + m)/(data['rev_operating'] + working_capital + m)

  # Fixed assets are long term assets (according to the internet)

  fixed_asset_turnover = (data['rev_operating'] + m)/(data['rev_operating'] + fixed_assets + m)

  # # Adding extracted features to data:
  # # Un comment all the below lines to add the extracted features into dataframe

  # data['current_liabilities'] = current_liabilities
  # data['current_assets_minus_inventory'] = current_assets_minus_inventory
  # data['cash_mktbl_securities'] = cash_mktbl_securities
  # data['working_capital'] = working_capital
  # data['fixed_assets'] = fixed_assets
  # data['current_ratio'] = current_ratio
  data['quick_ratio'] = quick_ratio
  # data['cash_ratio'] = cash_ratio
  # data['cfo_ratio'] = cfo_ratio
  # data['working_capital_turnover'] = working_capital_turnover
  data['fixed_asset_turnover'] = fixed_asset_turnover

  data = data.replace(np.nan,1)
  # new_features = ['current_liabilities','current_assets_minus_inventory','cash_mktbl_securities','working_capital',
  #                 'fixed_assets','current_ratio','quick_ratio','cash_ratio','cfo_ratio','working_capital_turnover',
  #                 'fixed_asset_turnover']
  new_features = ['quick_ratio','fixed_asset_turnover']
  relevantAttributes = list(relevantAttributes)
  relevantAttributes.extend(new_features)
  relevantAttributes = np.asarray(relevantAttributes)
  # print(relevantAttributes)

  return data,relevantAttributes

# Harness for Training

## Preprocessor

In [5]:
def preprocessor(df, relevantAttributes, new=False):
    df = df.replace(np.nan, 0) # replace all NaN with 0

    # Replacing values with absolute values
    for key in df.keys()[8:]:
      if key not in ['AR','profit','roa','roe','wc_net','ebitda','prof_financing','prof_operating']:
        df[key] = np.abs(df[key])

    if new == False: # model building, need to create target variable
      df['stmt_date'] = pd.to_datetime(df['stmt_date'], format='%Y-%m-%d') 
      df['def_date'] = pd.to_datetime(df['def_date'], format='%d/%m/%Y')  

      df['default'] = np.where(df['fs_year']== df['def_date'].dt.year-1, 1, 0) # create default target 

      April_prior = df[df['def_date'].dt.month <= 4] # yes shift
      April_after = df[(df['def_date'].dt.month > 4) | (pd.isnull(df['def_date'].dt.month))] # no shift
      April_prior[['def_date','default']] = April_prior[April_prior['def_date'].dt.month <= 4].groupby('id')[['def_date','default']].shift(-1) # shift
      df = pd.concat([April_prior, April_after]) # recombine

      df = df.dropna(subset = ['default']) # drop rows due to nulls being created by shift 

    new_df = df

    # load saved mean/std parameters
    with open(DATA_PATH+'parameters.pkl', 'rb') as f:
      parameters = pickle.load(f)

    # outlier handling
    for key in relevantAttributes:
      zscores = np.asarray(new_df[key] - parameters[key]['mean'])/parameters[key]['std']
      # print(np.sum(np.isnan(zscores)))
      outliers = (zscores > 1) + (zscores < -1)
      new_df[key] = new_df[key]*(~outliers) + new_df[key]*(outliers)/100


    # Normalization
    for key in relevantAttributes:
      new_df[key] = (new_df[key] - parameters[key]['mean'])/parameters[key]['std']

    # Extract features 
    new_df,relevantAttributes = feature_extraction_modified(new_df, relevantAttributes)
    # print(relevantAttributes)

    return(new_df, relevantAttributes)

## Estimator

In [6]:
def estimator(df, relevantAttributes, fitting_algo, loss_fn = 'binary_crossentropy'):
    if fitting_algo == neural_net_model:
      model = fitting_algo(df,relevantAttributes, loss_fn)
    else:
      model = fitting_algo(df, relevantAttributes)
    return(model)

## Models

### Logit

In [7]:
def logit_estimation_fn(df, relevantAttributes):
    relevantAttributes = " + ".join(relevantAttributes)
    model_logit = smf.logit(formula=f'default ~ {relevantAttributes}', data=df).fit()
    return(model_logit)

### Random Forest

In [8]:
def random_forest(df, relevantAttributes):
    y = df['default']
    X = df[relevantAttributes]
    clf = RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_split=10, random_state=0).fit(X,y)
    return(clf)

### Neural Net

In [9]:
def neural_net_model(df,relevantAttributes, loss_fn = 'binary_crossentropy'):
  Y = df['default']
  X = df[relevantAttributes]

  n_dims = len(relevantAttributes)

  model = Sequential()
  model.add(Input((n_dims)))
  model.add(Dense(16, activation= 'relu'))
  model.add(Dense(8, activation= 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(loss = loss_fn, optimizer = 'adam', metrics = [tf.keras.metrics.Recall(),tf.keras.metrics.Accuracy(name="accuracy", dtype=None),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.FalsePositives()])
  
  model.fit(X, Y, epochs=3)
  
  return model

## Predictor

In [10]:
def predictor(new_df, relevantAttributes, model):
    predictions = model.predict(new_df[relevantAttributes])
    return(predictions)

## Walk-forward harness for tuning and cross validation

In [11]:
def walk_forward_harness(df, relevantAttributes, preprocessor, estimator, predictor, model_type, 
                         start, date_col = 'fs_year', step_size = 1, loss_fn = 'binary_crossentropy'):     
    predictions = []
    model_list = []
    actual = []
    stats_list = {key: [] for key in ['auc','fpr','tpr','threshold']}
    
    max_year = df[date_col].max()
    
    tempAttributes = np.copy(relevantAttributes)

    for count in np.arange(0,int((max_year - start)/step_size - 1)): 
        # print(count) 
        relevantAttributes = np.copy(tempAttributes)
        
        # preprocess train set 
        train = df[df[date_col] <= start+count]
        train, _ = preprocessor(train, relevantAttributes) 

        # preprocess test set        
        test = df[df[date_col] == start+count+step_size+1]
        test, relevantAttributes = preprocessor(test, relevantAttributes)
        
        # fit model and predict 
        model = estimator(train, relevantAttributes, model_type, loss_fn = loss_fn)
        pred_prob = predictor(test, relevantAttributes, model)
        pred_prob = np.reshape(pred_prob, (test.shape[0]))

        # calculate statistics
        pred_df = pd.DataFrame({'actual': test['default'], 'pred_prob': pred_prob})
        auc = roc_auc_score(pred_df['actual'], pred_df['pred_prob'])
        fpr,tpr,threshold = metrics.roc_curve(pred_df['actual'], pred_df['pred_prob'])
 
        model_list.append(model)
        predictions.append(pred_prob)
        actual.append(pred_df['actual'])
        stats_list['auc'].append(auc)
        stats_list['fpr'].append(fpr)
        stats_list['tpr'].append(tpr)
        stats_list['threshold'].append(threshold)
    
    return(predictions, model_list, stats_list, actual)

# Validation

In [12]:
def validation(stats_list, actual, predictions):
  # average AUC
  print(f"Average AUC: {np.mean(stats_list['auc'])}")

  # optimal threshold
  optimal_idx = np.argmax(stats_list['tpr'][-1] - stats_list['fpr'][-1])
  optimal_threshold = stats_list['threshold'][-1][optimal_idx]

  # confusion matrix, precision, recall, accuracy
  y_true = np.asarray(actual[-1])
  y_preds = np.asarray(predictions[-1])
  confusion = confusion_matrix(y_true,y_preds>optimal_threshold)
  print("confusion matrix:")
  print(confusion)
  print("precision: {}".format(precision_score(y_true,y_preds>optimal_threshold)))
  print("recall: {}".format(recall_score(y_true,y_preds>optimal_threshold)))
  print("accuracy: {}".format(accuracy_score(y_true,y_preds>optimal_threshold)))

  # ROC curves
  for i in range(len(stats_list['fpr'])): 
    plt.plot(stats_list['fpr'][i],stats_list['tpr'][i])
    plt.title('ROC Curve')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

# Model Testing

## Logit

In [13]:
# relevantAttributes = ['asst_tot','eqty_tot','profit','ebitda','roa']
# predictions, model_list, stats_list, actual = walk_forward_harness(data, 
#                                                            relevantAttributes, 
#                                                            preprocessor, 
#                                                            estimator, 
#                                                            predictor, 
#                                                            logit_estimation_fn, 
#                                                            start=data['fs_year'].min())

In [14]:
# validation(stats_list, actual, predictions)

In [15]:
# for model in model_list:
#   print(model.summary())

## Random Forest

In [16]:
# relevantAttributes = ['asst_tot','eqty_tot','profit','ebitda','roa']
# predictions, model_list, stats_list, actual = walk_forward_harness(data, 
#                                                            relevantAttributes,
#                                                            preprocessor, 
#                                                            estimator, 
#                                                            predictor, 
#                                                            random_forest, 
#                                                            start=data['fs_year'].min())

In [17]:
# validation(stats_list, actual, predictions)

## Neural Net

In [18]:
# relevantAttributes = ['asst_tot','eqty_tot','profit','ebitda','roa']
# predictions, model_list, stats_list, actual = walk_forward_harness(data, 
#                                                            relevantAttributes, 
#                                                            preprocessor, 
#                                                            estimator, 
#                                                            predictor, 
#                                                            neural_net_model, 
#                                                            start=data['fs_year'].min())

In [19]:
# validation(stats_list, actual, predictions)

# Walk-forward harness to train on all data and pickle

In [20]:
def train_walk_forward_harness(data, relevantAttributes, preprocessor, predictor, model_type):
  train, relevantAttributes = preprocessor(data, relevantAttributes)
  model = estimator(train, relevantAttributes, model_type)

  with open(DATA_PATH+'model_final.pkl', 'wb') as f:
    pickle.dump(model, f)

  return model   

In [21]:
relevantAttributes = ['asst_tot','eqty_tot','profit','ebitda','roa']
model = train_walk_forward_harness(data, relevantAttributes, preprocessor, predictor, neural_net_model)

Epoch 1/3
31836/31836 [==============================] - 11s 326us/step - loss: 0.0514 - recall: 1.8772e-04 - accuracy: 0.0000e+00 - false_negatives: 10652.0000 - false_positives: 193.0000
Epoch 2/3
31836/31836 [==============================] - 10s 321us/step - loss: 0.0486 - recall: 0.0048 - accuracy: 0.0000e+00 - false_negatives: 10603.0000 - false_positives: 67.0000
Epoch 3/3
31836/31836 [==============================] - 10s 322us/step - loss: 0.0484 - recall: 0.0072 - accuracy: 0.0000e+00 - false_negatives: 10577.0000 - false_positives: 85.0000


# Final walk-forward harness

In [22]:
def test_walk_forward_harness(model, test_df, relevantAttributes, preprocessor, predictor):     
    test_df, relevantAttributes = preprocessor(test_df, relevantAttributes, new=True)

#     with open(DATA_PATH+'model_final.pkl', 'rb') as f:
#         model = pickle.load(f)
    
    pred_prob = predictor(test_df, relevantAttributes, model)
    pred_prob = np.reshape(pred_prob, (test_df.shape[0]))
    pred_prob[np.isnan(pred_prob)] = 0.5 # just in case

    return pred_prob

In [26]:
# holdout_sample = pd.read_csv('holdout_sample.csv')
holdout_sample = pd.read_csv('./data/train_demo_subset.csv')
relevantAttributes = ['asst_tot','eqty_tot','profit','ebitda','roa']
pred_prob = test_walk_forward_harness(model, holdout_sample, relevantAttributes, preprocessor, predictor)
holdout_sample.shape, pred_prob.shape

32/32 [==============================] - 0s 369us/step


((1000, 44), (1000,))

# Small example of what the data looks like:

In [30]:
holdout_sample.iloc[np.random.randint(10,200,5), [1,5]] # Cannot show full data due to privacy issues

,id,ateco_sector
194,13790282,28
177,27270248,32
109,1101350054,41
18,25150236,46
155,14510879,45


In [32]:
list(holdout_sample.columns)

['Unnamed: 0',
 'id',
 'stmt_date',
 'HQ_city',
 'legal_struct',
 'ateco_sector',
 'def_date',
 'fs_year',
 'asst_intang_fixed',
 'asst_tang_fixed',
 'asst_fixed_fin',
 'asst_current',
 'AR',
 'cash_and_equiv',
 'asst_tot',
 'eqty_tot',
 'eqty_corp_family_tot',
 'liab_lt',
 'liab_lt_emp',
 'debt_bank_st',
 'debt_bank_lt',
 'debt_fin_st',
 'debt_fin_lt',
 'AP_st',
 'AP_lt',
 'debt_st',
 'debt_lt',
 'rev_operating',
 'COGS',
 'prof_operations',
 'goodwill',
 'inc_financing',
 'exp_financing',
 'prof_financing',
 'inc_extraord',
 'taxes',
 'profit',
 'days_rec',
 'ebitda',
 'roa',
 'roe',
 'wc_net',
 'margin_fin',
 'cf_operations']